In [43]:
#The objective of this script is to extract metadata from tiff files that contain images of cells

# 1- I want to open tiff files that are in a specific folder and extract the metadata
#each tiff files has several frames
## for this i will use the package tifffile and the function imread to open the tiff files
import tifffile
from tifffile import imread

#I want to open a folder of tiff files and read each file
import os 
path = 'Drug1_1000nM/' #this is the path to the folder where the tiff files are
files = os.listdir(path) #this will list all the files in the folder 
for file in files: #this will loop over all the files in the folder
    open_tiff = imread(path + file) #this will open the tiff file
    
#i have to define the metadata that i want to extract from the tiff files

metadata = {'frame_number':[], 'cell_number':[], 'number_of_viable_cells':[], 'cell_size':[], 'nucleus_size':[], 'protein_concentration':[], 'number_of_mRNA_in_nucleus':[], 'number_of_mRNA_in_cytoplasm':[], 'number_of_transcription_sites':[], 'intensity_of_transcription_sites':[]}

# 2- The images have 3 channels, I want to extract specific data from each channel so i want to separate the channels
# channel 0: red (1) where we can see the nucleus
# channel 1: green (2) where we can see the protein expresion
# channel 2: blue (3) where we can see the mRNA and the transcription sites

## for this i will use the function split to separate the channels
import numpy as np
split_channels = np.split(open_tiff, 3, axis=0)

## I will use the function imshow to visualize the images and the channels separately 
import matplotlib.pyplot as plt
plt.imshow(split_channels[0])
plt.imshow(split_channels[1])
plt.imshow(split_channels[2])

# 3- I want to save the metadata in a csv file
## for this i will use the package pandas and the function to_csv to save the metadata in a csv file
import pandas as pd
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv('path/to/csv/file')

# 4- The metadata should have this information for each tiff image:

# column0: frame number

# column1: cell number

# column2: number of viable cells (this is 1 for alive cells and 0 for dead cells) in each frame and we are going to use channel 0
## for this i will use the function threshold to separate the cells from the background or i can use mask to separate the cells from the background 

# column3: cell size in each frame, we are going to use channel 1 to extract the cell size
##for this i can use mask to separate the cells from the background and then extract the cell size

# column4: nucleus size in channel 0 for each cell in each frame
## for this i can use mask to separate the cells from the background and then extract the nucleus size

# column5: protein concentration in channel 1 for each cell in each frame
## for this i can use mask to separate the cells from the background and then extract the protein concentration

# column6: number of mRNA in nucleus in channel 2 for each cell in each frame
## i can use mask and threshold to separate the nucleus from the background and then extract the number of mRNA in the nucleus

# column 7: number of mRNA in cytoplasmn in channel 2 for each cell in each frame
## i can use mask and threshold to separate the cytoplasm from the background and then extract the number of mRNA in the cytoplasm

# column7: number of transcription sites in channel 2 for each cell in each frame

# column8: intensity of transcription sites in channel 2 for each cell in each frame


"\n## for this i will use the function split to separate the channels\nimport numpy as np\nsplit_channels = np.split(open_tiff, 3, axis=0)\n\n## I will use the function imshow to visualize the images and the channels separately \nimport matplotlib.pyplot as plt\nplt.imshow(split_channels[0])\nplt.imshow(split_channels[1])\nplt.imshow(split_channels[2])\n\n# 3- I want to save the metadata in a csv file\n## for this i will use the package pandas and the function to_csv to save the metadata in a csv file\nimport pandas as pd\nmetadata_df = pd.DataFrame(metadata)\nmetadata_df.to_csv('path/to/csv/file')\n\n# 4- The metadata should have this information for each tiff image:\n\n# column0: frame number\n\n# column1: cell number\n\n# column2: number of viable cells (this is 1 for alive cells and 0 for dead cells) in each frame and we are going to use channel 0\n## for this i will use the function threshold to separate the cells from the background or i can use mask to separate the cells from th